# **05. 신규 지표 개발**
----

## **목적** 
* ### 앱 사용성을 보다 정확히 트래킹할 수 있는 신규 지표 로직/산출식을 개발한다.
* ### KUI (Key Usage Indicator)
## **목차**
* ### Usage 관련 EDA 진행
* ### Activation 기준 정의
* ### 지표 개발 로직 도출
* ### 모델링을 통한 지표 수식 개발
* ### 대시보드 구축


## **배경**
### 서비스의 사용성을 가장 간단하게 파악할 수 있는 지표가 DAU, WAU, MAU이다.
* ### DAU: Daily Active User
* ### WAU: Weekly Active User
* ### MAU: Montly Active User

### 기간에 대한 기준은 비교적 명확하나, Active 판단 기준은 상황에 따라 매우 다르게 정의된다. 이번 사례의 경우, 프로그램을 실행만 시켜도 Active로 판단한다고 가정해보자. 이런 방식이 틀린 방식은 아니지만, 만약 프로그램을 실행만 시키고 아무런 행동도 하지 않아도 해당 지표에 영향을 미치므로 허수가 포함된다. 또 개념상 정확한 Activation 으로 판단하기에 무리가 있어, 서비스 이용 패턴을 고려해 보다 정밀한 Activation 지표를 개발할 필요가 있다

* ### 현재 Active 판단 기준이 허술한 상황
* ### 기존에 구축된 대시보드가 매우 많고 중요하지 않은 대시보드도 많아 유지보수에 불필요한 리소스 낭비중
* ### 이용 패턴 및 요인별 중요도 등을 고려해 새로운 Activness 지표 개발 및 트래킹 필요

![screenshot](.\img\41.png "41_Sh")

## **목적**
### 이번 케이스의 목적은 명확하게 아래와 같이 정리된다.

* ### 정밀한 Activness를 대변하는 하나의 인덱스 산출 (OMTM)
* ### OMTM
    * ### Facebook: 7 friends within 10 days of signing up
    * ### Dropbox: put at least one file in one folder on one device
    * ### Twitter: visiting at least 7 times a month
* ### 인덱스 산출을 위한 변수와 가중치 도출     
    ![screenshot](.\img\42.png "42_Sh")

![screenshot](.\img\43.png "43_Sh")

# **05-2.분석 프레임**
-----

### 이번 케이스의 경우, 주 목적을 달성하기 위한 세부 프레임을 아래와 같이 정리할 수 있다.

## **프레임**

* ### KUI (Key Usage Index) 도출
    * ### Activness 데이터 EDA
    * ### Feature Selection
    * ### 변수별 Weight 및 산출식 도출

![screenshot](.\img\44.png "44_Sh")

## **Dataset**
* ### 일별 Active 유저수 (국가별, 기준별, 플랫폼별 구분)
* ### 특정 2주간의 이용행동 데이터 (2주 Retention 여부로 feature selection)
* ### Retention에 대한 영향력(coefficient)을 기준으로 가중치 산출

## **추가 고려사항**
### 지금 케이스와 같이 신규 지표를 개발하는 경우, 실제 지표를 트래킹/모니터링하고 대응 등 Action을 진행할 유관자와의 협업이 중요하다. 특히 유관자에게 지표의 의미와 영향력에 대한 공감과 이해를 제공해야 효과를 볼 수 있다. 상시적으로 유관자에게 프로젝트 배경, 목적에 대해 상기시키고 도메인 지식이 필요할 때 조언과 아이디어를 구하는 등 적극적인 태도가 요구된다.

* ### 일정 및 공유 방식
* ### 협업 엔지니어 및 담당자 등 배정
* ### 대시보드/모니터링 방식  

# **05-3. 데이터 탐색(1): 판정 기준 선택 & 패턴 이해**
---

### 데이터 탐색 과정을 통해 얻고자 하는 목적은 아래와 같다.

* ### 기존 DAU 를 대체할 수 있고 보다 정확한 Active 판정 기준은 무엇인가?
* ### 가설적인 여러 판정 기준 중 가장 적합한 기준은 무엇인가?
* ### 범주변인 (예, 국가별, 기준별, 플랫폼별, 결제유무 등)으로 구분했을 때, Activeness 패턴이 어떻게 달라지는가?

## **가설적 판정 기준**

### 배경에서 설명한 대로 단순히 DAU라는 지표를 통해 활동성을 트래킹하고 있는 상황이므로 앱을 실행하기만 해도 허수가 잡히게 된다. 따라서 가설적으로 몇 개의 Activeness 판정 기준을 설정해 데이터를 추출하고 탐색을 위한 준비를 진행할 필요가 있다. 가설적 판단 기준을 정의할 때 도메인 전문가의 지식을 이용하는 것이 효과적이다.

* ### option 1. 파일을 View 했을 경우 Active 로 판정
* ### option 2. 파일을 View and (Edit or Export) 했을 경우 Active 판정

## **Dataset**

### 이번에 추출된 데이터셋의 경우, 일반적인 경우와 조금 다르게 범주변인의 flag 를 잘 활용하여 분석을 진행해야 한다. 우선 데이터셋을 불러와서 데이터의 형태를 이해하도록 하자.

## **데이터 탐색(1) 판정 기준 선택**
### ```pd.read_table```명령어를 통해 데이터를 읽어오고, ```head()```, ```info()``` 함수를 통해 데이터 형상을 확인하도록 하자.

* ### 일별 Active 유저수 카운트 (국가별, 기준별, 플랫폼별 구분)
* ### 데이터 포멧이 일반적인 포멧과 다름 (대부분 카테고리 변수, 하나의 컬럼만 집계된 결과)
* ### if dataType == 'all', View + Edit + Export
* ### if dataType == 'view', View only
* ### if values = 'ALL': 모든 범주변인의 합산
* ### all and ALL are different values

## **전처리**
### 아래와 같은 간단한 전처리 과정을 진행한다.
```
# slicing based on row numbers
df = df.ix[:len(df)-4]

# timestamp to pandas date format
df['datetime'] = pd.to_datetime(df['datetime'], unit='s').dt.date # timestamp to datetime of pd

# filtering
df = df[df['countryCode'] != '--']
```

## **변수 탐색**
### 전처리가 완료되었다면, 데이터셋을 이해하기 위해서 각 변수별 유니크 값과 각 빈도 혹은 분포를 살펴보는 것이 가장 기본이다. 범주 변수일 경우 ```value_counts()```, 연속 변수일 경우 ```describe()``` 함수를 통해 간단히 확인 가능하다.
```
# freqeuncy table
df.countryCode.value_counts()

# distribution
df.userCount.describe()
```

## **필터링**
### 마지막으로 필터링을 통해, 목적에 맞게 데이터셋을 구분하는 작업이 필요하다. Pandas 데이터프레임에서 쓰이는 필터링 함수는 두 가지이다.

* ### ```df[df['col_name'] == '조건']```
* ### ```df.query("col_name == '조건")```
```
## df all-base active
df_all = df.query("dataType == 'all'")

## all base total
df_all_tot = df_all.query("deviceTypeNo == 'ALL' & osTypeNo == 'ALL' & aswTypeNo == 'ALL' & isPaid == 'ALL' & countryCode == 'ALL'")
```

## **시각화**
### 여러 조건을 기준으로 데이터 프레임을 구분하고, Activeness 산출 결과에 대한 시각화를 진행하자.
```
plt.figure(figsize=(12,7))

df_all_tot.groupby("datetime")['userCount'].sum().plot(marker='o', label='all')
df_view_tot.groupby("datetime")['userCount'].sum().plot(marker='*', label='View')
dau.plot(marker='>', linestyle='dashed', label='Original_DAU')

plt.title("Daily Active Users")
plt.ylabel("Unique User Count")
plt.ylim(400000, 1500000)
plt.legend()
plt.tight_layout()
```

![screenshot](.\img\45.png "45_Sh")

### 위와 같은 과정을 통해 판정 조건별 Activness 지표를 산출해보고, 기존의 지표와 비교하고 논의함으로써 하나의 판정 기준을 정하도록 한다.

## **데이터 탐색(2) Activeness 세분화**
### 사전적으로 분석(分析)이란, 얽혀 있거나 복잡한 것을 풀어서 개별적인 요소나 성질로 나눔으로 정의된다. 즉 집계된 하나의 지표를 MECE 철학에 근거해 최대한 쪼개고 나눔으로써 그 본질을 이해하는 것이 분석 과정이라 할 수 있다.

## 새로운 판정 기준으로 선택된 Activeness 지표를 여러 범주 변인을 기준으로 세분화함으로써 지표에 대한 이해를 높일 필요가 있다. 데이터셋의 주요 범주 변인은 아래와 같다. 해당 범주 변인들은 독립적으로 구분되지만, 서로 상호작용 효과를 살펴볼 수도 있다.

* ### osType: Andriod, iOS, Win, Mac
* ### isPaid: Free, Paid
* ### countryCode: KR, US ...

## **함수 생성 및 필터링**
### 만약 거의 동일한 조건을 반복해서 실행할 경우 함수를 생성하여 작업하는 것이 효율적이다.

```
def df_by_ostype(df, ostype, excl = 'ALL'):

    df = df[(df['osTypeNo'] == ostype) & (df['deviceTypeNo'] == excl) \
            & (df['aswTypeNo'] == excl) & (df['isPaid'] == excl) & (df['countryCode'] == excl)]

    return df
```

## **시각화**

### 결과에 대한 직관적인 이해를 돕기 위해 아래와 같이 시각화를 진행하도록 하자. 가능한 상세하고 반복적으로 탐색을 진행하여 데이터에 대한 이해를 높이는 것이 필요하다.

```
fig = plt.figure(figsize=(12,7))

ax = fig.add_subplot(111)
df_aos.groupby("datetime").sum()['userCount'].plot(label='Android', ax=ax, linestyle='dashed', c='b')

ax2 = ax.twinx()
df_ios.groupby("datetime").sum()['userCount'].plot(label='ios', ax=ax2, c='k', linestyle='dashed')
df_win.groupby("datetime").sum()['userCount'].plot(label='Win', ax=ax2, c='g', linestyle='dashed')
df_mac.groupby("datetime").sum()['userCount'].plot(label='Mac', ax=ax2, c='y', linestyle='dashed')

plt.title("Daily Active Users by Platform")
ax.legend(loc=2)
ax2.legend(loc=1)
plt.tight_layout()
```

![screenshot](.\img\46.png "46_Sh")

```
# Daily Active User by Country
df_cont_pivot[cont_idx[1:]].plot(figsize=(9,6));

plt.title("Daily Active Paid Users by top 10 Country")
plt.grid(color='lightgrey', alpha=0.5, linestyle='--')
plt.legend()
plt.tight_layout()
```

![screenshot](.\img\47.png "47_Sh")

## **Summary**

* ### 기존 DAU에 허수값 존재: DAU -> all(View+Edit+Export)으로 활동성 지표 변경 고려
* ### 활동성은 안드로이드가 절대적으로 높음 (Win > iOS > Mac)
* ### 무료 이용자 비율이 매우 높음 (특히 안드로이드)
* ### 유료 이용자의 경우 윈도우(PC) 사용 경향이 높은편
* ### 국가별 분석결과, 무료 및 유료 이용자 모두 한국이 많음 (유료의 경우 미국이 특히 높음)

[실습파일](https://github.com/songhunhwa/songhunhwa.github.com/tree/master/tutorial/tutorial_03 "데이터 탐색")

# **05-4. 데이터 탐색(2): 변수 탐색 및 선정**
-----

### 1차 데이터 탐색을 통해 Activeness 판정 기준을 선정했다면, 해당 판정 기준으로 변수별 가중치를 산출하기 위한 과정을 진행할 필요가 있다. 비록 새로운 판정 기준으로 일별 Activeness를 카운트해 지표를 개선할 수 있으나, 보다 상세하고 정확한 지표를 개발하기 위해 이용 관련 변수를 추출하고 특정 이벤트나 행동 (예, 잔존 여부)을 기준으로 가중치를 산출해 신규 지표 (KUI)를 개발하는 과정을 진행하도록 하자.

## **산출식**
### 일반적으로 지표는 아래와 같이 가중치 X 변수의 합으로 정의한다.

![screenshot](.\img\48.png "48_Sh")

## **변수 선정**
### 변수 후보를 선택할 때는 도메인 영역의 지식을 활용해 아래와 같이 후보군을 추릴 수 있다. 예측하고자 하는 변수 역시 동일한 방식으로 선택하는 것이 필요하다.

* ### 예측변수
    * ### 특정 7일간, 이용 파일수(유니크), 총 열람 횟수 및 저장횟수, 내보내기 횟수
    * ### 특정 7일간, 열람 트래픽, 편집 트래픽, 내보내기 트래픽
    * ### 특정 7일간, 주간 방문횟수 (FQ7)
* ### 종속변수
    * ### 다음주 잔존 여부 (Boolean, 0 or 1)

## **Dataset**
### 이번 분석을 위해 필요한 데이터셋은 아래와 같다.

* ### 특정 7일간, 유저별 사용행동 횟수 (독립변수)
* ### 다음주 유저별 사용행동 횟수 (예측변수)
### 위 2개의 데이터셋의 형상은 동일하다(날짜의 차이만 존재). 단 두번째 데이터셋의 iduser를 기준으로 첫번째 데이터셋과 Left Join하여 첫번째 데이터셋에 하나의 추가된 컬럼(잔존 여부)을 만들어 예측변수로 사용할 수 있다.

## **데이터탐색(1) 독립변수 탐색**
### 마찬가지로, ```pd.read_csv``` 를 통해 데이터를 읽어오고, ```head()``` 및 ```tail()```, ```info()``` 명령어를 통해 간단히 데이터를 탐색해보도록 하자. 유저식별값만 수치형이 아닌 범주형이므로 ```set_index()```를 통해 pandas 데이터프레임에서 인덱스로 처리하는 것이 추후에 편리하다.

## **전처리**
### 이번 케이스와 같이 수치형으로만 구성된 데이터셋의 경우 ```describe()```를 통해 간단히 분포 확인 및 이상치를 확인할 수 있다. 이번 데이터셋의 경우, 아웃라이어 처리외에 아래와 같은 전처리가 필요하다.

* ### 인덱스 처리
* ### 단위 변환
* ### 아웃라이어 제거
* ### 파생변수 생성
* ### 테이블 조인

### 우선 dataframe의 인덱스를 유니크 값으로 설정하면, 추후 모델링 과정에서 이 컬럼을 빼는 작업을 할 필요가 없고 조인 등 다른 전처리 과정에서 유용하게 사용할 수 있다.

```
# 인덱스 지정 컬럼이 유니크한지 확인
df.iduser.is_unique # duplications check

# 유니크한 컬럼을 dataframe의 인데스로 지정
df.set_index("iduser", inplace=True)
```

### 원본 데이터의 트래픽의 경우 단위가 byte이다. 수치 자체가 매우 큰 반면 제공하는 정보 수준은 낮으므로, megabyte 수준으로 단위를 변환하는 것이 유용하다.

```
# byte -> mb 로 변환하는 예제
tf.viewtraffic = (tf.viewtraffic/1024/1024).round(0).astype(int)
```

### 항상 존재하는 아웃라이어는 표준편차 혹은 IQR방식을 활용해 제거하는 것이 좋다.

```
# z-score 기준 +- 3을 벗어나는 값을 제거하는 예시
def remove_outlier_std(df):
    for i in range(0, len(df.iloc[1])):
        df.iloc[:,i] = df.iloc[:,i].replace(0, np.NaN)
        df = df[~(np.abs(df.iloc[:,i] - df.iloc[:,i].mean()) > (3*df.iloc[:,i].std()))]
    return df.fillna(0)
```

### 경우에 따라, 비즈니스 상황을 고려해 창의적으로 파생변수를 생성하는 것이 모델의 성능을 올리는 데 큰 도움을 줄 수 있다. 다중공선성만 유의하면서 최대한 창의적으로 신규 변수를 생성해보는 것이 필요하다.

```
# 유관 변수를 활용해 새로운 파생변수 생성
df2['totalcount'] = df2['filecount'] + df2['savecount'] + df2['exportcount']
```

### 실무에서 대부분의 경우 한 개의 테이블로 분석 가능한 상태가 되는 경우는 드물다. 따라서 여러 개의 테이블을 특정 키를 기준으로 조인해 데이터를 처리하는 과정을 반복해서 진행한다.

```
# pandas dataframe 을 조인하는 예시, join key == index
df3 = pd.merge(df2, tf_2nd_wk['flag'].to_frame(), left_index=True, right_index=True, how='left').fillna(False)
```

## **변수별 영향력 도출**
### 아래와 같은 과정으로 여러 변수중에 유의미한 변수를 선택/추출한다.

* ### 데이터 표준화 (MinMax Scaling or Z-score)
* ### 회귀모형의 회귀계수 (Logistic Regression)
* ### 의사결정모형의 Feature Importance
* ### RFE 방식
```
# 데이터 표준화 (MinMax scaling)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
```
```
# 로지스틱 회귀를 이용한 coefficient 도출
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

log_rg = LogisticRegression().fit(X_train_scaled, y_train)
cross_val_score(log_rg, X_train_scaled, y_train, cv=5)
log_rg.score(X_test_scaled, y_test)
print(classification_report(y_test, log_rg.predict(X_test_scaled)))

feature_importance_lr = pd.DataFrame(zip(X.columns.values, log_rg.coef_.ravel()))
feature_importance_lr.columns = ['feature', 'coef']
feature_importance_lr.sort_values("coef", ascending=False, inplace=True)
```

![screenshot](.\img\49.png "49_Sh")

```
# 랜덤포레스트를 이용한 영향력 도출
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10).fit(X_train_scaled, y_train)
cross_val_score(rf, X_train_scaled, y_train, cv=3)
rf.score(X_test_scaled, y_test)
print(classification_report(y_test, rf.predict(X_test_scaled)))

feature_importance_rf = pd.DataFrame(zip(X.columns.values, rf.feature_importances_))
feature_importance_rf.columns = ['feature', 'importance']
feature_importance_rf.sort_values("importance", ascending=False, inplace=True)
```

![screenshot](.\img\50.png "50_Sh")

```
# 반복적 변수 제거 방식을 이용한 변수별 중요도 도출
from sklearn.feature_selection import RFE

select = RFE(RandomForestClassifier(n_estimators=5, random_state=42), n_features_to_select = 5)
select.fit(X_train_scaled, y_train)

feature_importance_rfe = pd.DataFrame(zip(X.columns.values, select.get_support()))
feature_importance_rfe.columns = ['feature', 'importance']
feature_importance_rfe.sort_values("importance", ascending=False, inplace=True)
```

![screenshot](.\img\51.png "51_Sh")

# **05-5. 스코어 산출 및 대시보드 구축**
----
### 방문일수(visitdays)를 주요 변수로 선정해, 이 변수를 기준으로 유저를 그룹핑하고 그 다음주의 재방문 여부를 binary로 표시해 데이터를 재추출한다. 그룹별로 재방문여부에 영향을 미치는 정도를 분석하여, 이를 기반으로 선행지수를 산출하는 것이 주요 목적이다.

## **스코어 산출**
### **그룹별 재방문 여부 데이터셋**
* ### group: day를 기준으로 특정 한 주간 방문일수를 의미함
* ### isnextvisit: 그 그룹에 해당하는 유저가 다음주에 재방문을 했는지 여부를 표시함 (0: 재방문x, 1: 재방문o)
![screenshot](.\img\52.PNG "52_Sh")

## **전처리**
* ### 데이터 타입 변환
```
next_ret['isnextvisit'] = next_ret['isnextvisit'].astype(str)
```

* ### 더미코딩 및 조인
```
next_ret1 = pd.get_dummies(next_ret.group, prefix='FQ')\
              .join(next_ret.isnextvisit)\
              .join(next_ret.day)\
              .drop("FQ_1", axis=1)
```

## **모델 학습 및 가중치 산출**
* ### 여러 날짜의 데이터로 영향도를 구하고 평균 산출해 일관성 확보
* ### 가중치(영향도) 및 원점수를 이용해 one-single score 산출

```
from datetime import timedelta

dates = pd.to_datetime('2016-09-27')
for i in range(0, next_ret1['day'].nunique()):
    subset = next_ret1[next_ret1['day'] == dates]
    subset['intercept'] = 1.0

    X = subset.drop(['isnextvisit', 'day'], axis=1)
    y = subset.isnextvisit

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    logreg = LogisticRegression().fit(X_train, y_train)

    print(logreg.score(X_test, y_test))
    print(logreg.coef_.flatten())    
    dates = dates + timedelta(days=1)
```

## **변수별 가중치를 적용하여 스코어 산출**
```
daily_score = next_ret.groupby(['day', 'group']).size().unstack()

weight = [x for x in beta.mean()][:-1]
weight.insert(0, 1.0)

for i in range(1, 7):
    daily_score.iloc[:, i] = (daily_score.iloc[:, i] * weight[i-0]).apply(lambda x: round(x, 2))

daily_score['kui'] = daily_score.sum(axis=1)

# 스코어 시각화
daily_score['kui'].plot()
plt.grid(color='lightgrey', alpha=0.5, linestyle='--')
plt.tight_layout()
```
![screenshot](.\img\53.PNG "53_Sh")

## **대시보드 생성**
* ### Redash
* ### Plotly
    * ### line chart example
* ### creat a dashboard
